[View in Colaboratory](https://colab.research.google.com/github/akielbowicz/7506-organizacion_de_datos_fiuba/blob/master/TP1.ipynb)

#TIPO DE GRAFICO COPADO PARA HACER:
- swarpplot
- pairplot : https://www.youtube.com/watch?v=cpZExlOKFH4    (using correlation analysis)
- alguna serie de tiempo
- scatter plot
- Boxplot : 

#PREGUNTAS QUE NOS HACEMOS:

---
## Que Hacer
1. Limpiar datasets
2. Estudiar demografia : educacion, sexo, barrio
1. 





## Categorias a tener en cuenta

2. **ANALISIS DESCRIPTIVO**:
  1. Distribucion geografica de postulantes (zona vs idpostulante)
  2. Segmentar categoriaavisos vs sexo.
  3. distribucion categorias vs idaplicantes
  4.
3. estudiar etapas de postulacion: la postulacion es un paso posterior a la visita realizada. Entoces: 
  1. instancias de visitas: cuantas visitas se realizan sobre el mismo idaviso.
    1. cuantas visitas hago sobre el mismo idaviso para luego postularme.
  2. tasa de conversion  postulaciones: cuantas visitas se transforman en la postulacion.
  
4. **RATIOS DE RENDIMIENTO** : 
  1. cual es el % de aciertos en las aplicaciones? (de la cantidad de aplicaciones hechas, en cuantas avanzo?)
  2. %
  
  
  
  
  
[]

## Preguntas sencillas


1.   histograma de edades (fecha aplicacion - fecha nacimiento = edad)
2.   Elemento de la lista
3.   Que empresa tiene mas [avisos, visitas, postulaciones]
  
![Modelo de finanza] (http://aseduco.com/blog1/wp-content/uploads/2017/02/RESUMEN-RATIOS-FINANCIEROS.jpg)
![Modelo de finanzas II] (http://www.enciclopediafinanciera.com/images/indicadores-financieros-tipos.jpg)

## Tareas a completar


---


[] Normalizar las tablas

[] Limpiar los datos

[] Armar modelo estrella

[] Segmentar datos por:
1. Postulante
  * [] sexo
  * [] edad
  * [] nivel_educativo
2. Aviso:
  * [] fecha_postulacion
  * [] lugar_de_trabajo
  * [] nombre_area_laboral
  
3. Construir matriz con tasas de 


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1K4uRag5nmGtfuvzyJV9RL_73lzsh_iTO'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

downloaded.GetContentFile('datos_navent_fiuba.tar')
!tar -xvf datos_navent_fiuba.tar
!ls datos_navent_fiuba/

# !cat datos_navent_fiuba/TERMINOS.txt

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(context='notebook',color_codes=True)
pd.options.display.max_rows = 10

# Tabla de contenido

[Unidad 1: Limpieza de datos](#Unidad1)

[Unidad 2: Análisis exploratorio](#Unidad2)

**TOC solo para Colab**

[Unidad 1: Limpieza de datos](#scrollTo=vshkbKh5El4A)

[Unidad 2: Análisis exploratorio](#scrollTo=Pv0TcFXSCv_g)


[Seaborn CheatSheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Python_Seaborn_Cheat_Sheet.pdf)

# Unidad 1 : Limpieza de datos
<a id="Unidad1"></a>

##1.1 Importar Datos
Se cargan los datos y se redefinen los nombres de cada dataframe. Con esto logramos entender mejor los datos que tienen cada dataframe.


In [0]:
postulantes_educacion = pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_genero_edad = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
visitas = pd.read_csv('datos_navent_fiuba/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv')
avisos_online = pd.read_csv('datos_navent_fiuba/fiuba_5_avisos_online.csv')
avisos_detalle = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')


### 1.1.1 Renombramos Columnas
Nos permite trabajar mas comodamente, dado que referenciamos los atributos de manera intuitiva. Solo cambiamos las tablas que creemos que deben modificarse el nombre de alguna columna. En este caso: postulantes_educacion, visitas, avisos_detalle.

In [0]:
postulantes_educacion = postulantes_educacion.rename(index=str,columns={"idpostulante": "id_postulante", "nombre":"nivel_educacion"});
postulantes_genero_edad = postulantes_genero_edad.rename(index=str, columns={"idpostulante": "id_postulante", "fechanacimiento":"fecha_nacimiento"})
visitas = visitas.rename(index=str,columns={"idAviso":"id_aviso", "idpostulante": "id_postulante", "timestamp":"fecha_visita"});
avisos_detalle = avisos_detalle.rename(index=str, columns={"idaviso":"id_aviso", "titulo":"titulo_aviso", "descripcion":"descripcion_aviso","nombre_zona":"zona_de_trabajo","mapacalle":"direccion","nombre_area":"nombre_area_laboral","denominacion_empresa":"nombre_empresa"});
avisos_online = avisos_online.rename(index=str, columns={"idaviso":"id_aviso"})
postulaciones = postulaciones.rename(index = str, columns={"idaviso":"id_aviso", "idpostulante":"id_postulante", "fechapostulacion":"fecha_postulacion"})


In [0]:
# Agrego una columna de 1, en la tabla "avisos_detalle" . Me sirve para un analisis posterior.
avisos_detalle['dummy'] = 1


##1.2 Conocer Información de datos
Buscamos conocer el tipo de dato y su descripcion. Para ello creamos una funcion *describir_dataframe* para luego replicarlo para cada dataframe.


### 1.2.1Creamos funcion *describir_frame*

In [0]:
def describir_dataframe(df):
    '''Muestra caracteristicas '''
    print('-Información general:\n')
    df.info()
    print('\n- Descripción de las columnas:\n')
    print(df.describe(include='all'),'\n')
    print('- Muestra del DataFrame:\n')
    print(df.head(5))
    return

A partir de la descripción del dataframe vemos que cambios podemos realizar al mismo,
ya sea cambio de nombres, cambio de tipo de dato y control de valores nulos.

#### 1.2.1.1 Aplicamos funcion a : postulantes_educacion

In [0]:
describir_dataframe(postulantes_educacion)

*Para validar si aumentan los datos*
> 10/04 
Index: 298231 entries, 0 to 298230
Data columns (total 3 columns):
id_postulante      298231 non-null object
nivel_educacion    298231 non-null object
estado             298231 non-null object
dtypes: object(3)
memory usage: 9.1+ MB


Vemos que el dataframe tiene 3 columnas. Tanto *nombre* como *estado* son columnas categóricas
dado que toman pocos valores únicos, por lo cual podemos cambiarle el tipo de dato (de *object* a *category*).

Tambien se ve que hay **idpostulante** repetidos por lo que es necesario ver por qué motivo ocurre esto.

In [0]:
postulantes_educacion = postulantes_educacion.astype(dtype={'nivel_educacion':'category','estado':'category'})
postulantes_educacion.info()

Validamos que se haya realizado el cambio y vemos como la memoria se redujo a menos de la mitad.

#### 1.2.1.2 Aplicamos funcion a : **postulantes**_genero_edad





In [0]:
describir_dataframe(postulantes_genero_edad)

Hay tantos **idpostulante** como valores únicos, lo que indica que no hay *repetidos*.

In [0]:
postulantes_genero_edad = postulantes_genero_edad.astype(dtype={'sexo':'category'})
postulantes_genero_edad.info()

En el paso anterior no convertimos la columna *fechanacimiento* porque hay que hacer un análisis más detallado dado que las filas no tienen el mismo formato. 

In [0]:
print('Fechas ordenadas en orden ascendente, mostrando los nulos primero:\n')
print(postulantes_genero_edad['fecha_nacimiento'].value_counts(dropna=False).sort_index(na_position='first').head(10))
print('Fechas ordenadas en orden descendente:\n')
print(postulantes_genero_edad['fecha_nacimiento'].value_counts().sort_index(ascending=False).head(10))

In [0]:
print('Hay un {:.2f}% de valores nulos'.format(postulantes_genero_edad['fecha_nacimiento'].isna().mean() * 100))

**CONCLUSION** Al tener 2.36 % de los datos con NaN, notamos que no hay gran proporción que pueda afectar en los resultados futuros de análisis.

**DECISION** : Se decide mantener en el df los NaN values para no alterar la fuente y mostrar el coportamiento de las mismas en relación a los datos correctamente tomados.

Para saber si podemos tirar los datos que no tienen el formato correcto vemos el resto de las columnas

In [0]:
print(postulantes_genero_edad.sort_values(by=['fecha_nacimiento']).head(10),'\n')
print(postulantes_genero_edad.sort_values(by=['fecha_nacimiento'],ascending=False).head(1))


la primer fecha de nacimiento es [Council of War](https://founders.archives.gov/documents/Washington/03-01-02-0045)
[Lanus](https://encrypted.google.com/maps/place/C1092,+Av.+9+de+Julio+1775,+B1824KKQ+Lanús+Este,+Buenos+Aires/@-34.7125301,-58.3863253,17z/data=!3m1!4b1!4m5!3m4!1s0x95bccd3a19b52e5f:0x478bb3a3c0651908!8m2!3d-34.7125345!4d-58.3841366?hl=en)
[Ministerio de Salud](https://encrypted.google.com/maps/place/C1092,+Av.+9+de+Julio+1775,+C1092+CABA/@-34.612109,-58.3834105,17z/data=!3m1!4b1!4m5!3m4!1s0x95bccad0cc5fe421:0x5b5478bd8c558716!8m2!3d-34.6121134!4d-58.3812218?hl=en)
Anotamos el id_postulante para futuras exploraciones **xkdvwm0**

Similar para el más joven **a5qWAm**

Tranformamos las fechas con la función `to_datetime` con la opción `errors='coerce'` para que reemplace las fechas no válidas con `NaT`

In [0]:
postulantes_genero_edad['fecha_nacimiento'] = pd.to_datetime(postulantes_genero_edad['fecha_nacimiento'],format='%Y-%m-%d',
              errors='coerce')

print(postulantes_genero_edad['fecha_nacimiento'].value_counts(dropna=False).sort_index(),'\n')

postulantes_genero_edad.info()

Vemos que el número de filas nulas incrementó en el número correcto.

Definimos la tabla de postulantes como la union de postulantes_genero_edad y postulantes_educacion

In [0]:
postulantes = postulantes_educacion.merge(postulantes_genero_edad, left_on='id_postulante', right_on='id_postulante', how='outer')
print(postulantes)

#### 1.2.1.3 Aplicamos funcion a : visitas





In [0]:
describir_dataframe(visitas)

> 10/04 Index: 961897 entries, 0 to 961896
Data columns (total 3 columns):
id_aviso         961897 non-null int64
fecha_visita     961897 non-null object
id_postulante    961897 non-null object
dtypes: int64(1), object(2)
memory usage: 29.4+ MB

Algo raro en la descripción de las columnas es que no se muestra el número de valores únicos para 
**idAviso**. Por lo cual analizamos esta columna por separado

In [0]:
with pd.option_context('display.max_rows', 25):
  print(visitas['id_aviso'].value_counts().sort_index(),'\n');
  print(visitas['id_aviso'].value_counts().sort_values(),'\n');

Con eso vemos que hay 7458 avisos distintos 

In [0]:
with pd.option_context('display.max_rows', 20):
  print(visitas['fecha_visita'].value_counts().sort_index(),'\n');
  print(visitas['fecha_visita'].value_counts().sort_values());

In [0]:
fecha_visita = pd.to_datetime(visitas['fecha_visita'])
visitas['timestamp_visita'] = fecha_visita
visitas['fecha_visita'] = fecha_visita.dt.date
visitas['horario_visita'] = fecha_visita.dt.time
visitas.info()

In [0]:
# pandas_profiling.ProfileReport(visitas);

#### 1.2.1.4 Aplicamos funcion a : postulaciones


In [0]:
describir_dataframe(postulaciones)

> 10/04
<class 'pandas.core.frame.DataFrame'>
Index: 3401623 entries, 0 to 3401622
Data columns (total 3 columns):
id_aviso             int64
id_postulante        object
fecha_postulacion    object
dtypes: int64(1), object(2)
memory usage: 103.8+ MB

In [0]:
with pd.option_context('display.max_rows', 20):
  print(postulaciones['id_aviso'].value_counts(dropna=False).sort_index(),'\n');
  print(postulaciones['id_aviso'].value_counts().sort_values(),'\n');

In [0]:
fecha_postulacion = pd.to_datetime(postulaciones['fecha_postulacion'])
postulaciones['timestamp_postulacion'] = fecha_postulacion
postulaciones['fecha_postulacion'] = fecha_postulacion.dt.date
postulaciones['horario_postulacion'] = fecha_postulacion.dt.time
postulaciones.info()


#### 1.2.1.5 Aplicamos funcion a : avisos_online


In [0]:
describir_dataframe(avisos_online)

In [0]:
print(avisos_online['id_aviso'].value_counts().sort_values())

#### 1.2.1.6 Aplicamos funcion a : avisos_detalle


In [0]:
describir_dataframe(avisos_detalle)

> 10/04
Index: 13534 entries, 0 to 13533
Data columns (total 11 columns):
id_aviso               13534 non-null int64
idpais                 13534 non-null int64
titulo_aviso           13534 non-null object
descripcion_aviso      13534 non-null object
zona_de_trabajo        13534 non-null object
ciudad                 47 non-null object
direccion              872 non-null object
tipo_de_trabajo        13534 non-null object
nivel_laboral          13534 non-null object
nombre_area_laboral    13534 non-null object
nombre_empresa         13529 non-null object
dtypes: int64(2), object(9)
memory usage: 1.2+ MB

**ANALISIS** Busco entender la columna *ciudad* dado que veo algo raro. 

**CONCLUSION** Aplico scrip **pandas_profiling** (hace reporte del df).




*A* continuacion, 

In [0]:
avisos_detalle = avisos_detalle.astype(dtype={'idpais':'category','tipo_de_trabajo':'category',
                                             'nivel_laboral':'category','ciudad':'category',
                                             'nombre_area_laboral':'category'})

In [0]:
describir_dataframe(avisos_detalle)

Vemos que el  **idpais** es lo mismo para todos las filas por lo cual es un dato redundante y se puede eliminar.


In [0]:
avisos_detalle = avisos_detalle.drop(columns=['idpais'])

In [0]:
avisos_detalle['esta_activo'] = avisos_detalle['id_aviso'].isin(avisos_online['id_aviso'].values)

In [0]:
print(avisos_detalle['esta_activo'].value_counts(),'\n')
print('Un {:.2%} de los avisos  están activos.'.format(avisos_detalle['esta_activo'].mean()))

|postulante|
|----------|
|postulante_educacion|postulante_genero_edad|

|postulaciones|
|----|
|visitas|postulaciones|flag_postulacion|

|avisos|
|-----|
|aviso_online|aviso_detalle|flag_online|

In [0]:
#id = avisos_detalle[avisos_detalle['esta_activo'] == False]['id_aviso'].unique()
#print(id[:5])

# ES CRONOLOGICO LA FECHA DE VISITA CON LA PODSTULACIÓN?
Exploramos las fechas de postulacion y visitas, analizando maximos y minimos para entender la cronologia.  
### Cuadro de las fechas


Fecha| 15-Ene | 23-Feb | 28-Feb | 01-Mar 
:-- | :-:    | :-:    | :-:    | --:   
Visitas |--- | XXX | XXX | XXX
Postulaciones| XXX | XXX | XXX | ---

Nos damos cuenta que ciertas postulaciones ocurrieron previamente a la minima fecha de visitas. Esto suena inconsistente dado que no puede ocurrir! Pero dado que la meustra utilizada para hacer el trabajo es chica, podemos decir que las postulaciones incurren a las visitas realizadas anteriormente (y que no estan en el el dataset provisto por Navent.)

# Unidad 2: Análisis exploratorio
<a id="Unidad2"></a>

In [0]:
#Definimos 4 nuevos dataframes para que queden como backup de lo que tenemos 
#hasta ahora. Solo lo usamos mientras vamos armando los analisis, de manera
#de no tener que correr todo lo anterior cada vez que hay que nos equivoquemos.
if False:
  postulaciones_bkp = postulaciones.copy()
  visitas_bkp = visitas.copy()
  postulantes_bkp = postulantes.copy()
  postulaciones_bkp.info(), visitas_bkp.info(), postulantes_bkp.info()

# Para restaurar: Cambiar a True
if True:
  postulaciones = postulaciones_bkp.copy()
  visitas = visitas_bkp.copy()
  postulantes = postulantes_bkp.copy()

Comenzamos analizando la informacion de cada uno de los dataframes que tenemos.

## 2.1 Postulantes
Conozcamos a nuestros postulantes

In [0]:
postulantes.head()

Observar que el primer postulante aparece nuevamente con distinto nivel academico.
El segundo postulante incluso tiene varios estados.
Podemos hacer un grafico que tenga a cada postulante con nivel educativo y que muestre un punto si esta en cada una de las categorias

In [0]:
#Ver porcentaje por nivel educativo
#Porcentaje que cambio de nivel
#Hacer grafico que muestre de cada anio cuantos tiene cada nivel educativo (stackplot)
#Ver numero de postulaciones o visitas de cada postulante estando en cada nivel educativo
#Ver corelacion nivel educativo vs sexo
#Ver cantidad de postulante segun sexo por anio de nacimiento

In [0]:
nro_registros = postulantes.shape[0]
nro_postulantes = postulantes['id_postulante'].unique().size
porcentaje_postulantes_repetidos = 1 - nro_postulantes / nro_registros

print('Hay {:d} registros de postulantes de los cuales solo\n'\
      'hay {:d} postulantes distintos.\n'\
      'Es decir, hay un {:.2%} de '\
      'postulantes repetidos.\n'.format(nro_registros,nro_postulantes,\
                                       porcentaje_postulantes_repetidos))

print('Calculamos la cantidad y el porcentaje de valores nulos por columna:')
print(postulantes.isnull().agg(['sum','mean']).T)

Vemos que tres de las columnas contienen valores nulos. 

### 2.1.1 Educacion

In [0]:
# Grafico de porcentaje de nivel de educación de los postulantes
postulantes['nivel_educacion'] = ( postulantes['nivel_educacion'].cat
                                              .reorder_categories([
                                              'Otro',
                                              'Secundario',
                                              'Terciario/Técnico',
                                              'Universitario',
                                              'Master',
                                              'Posgrado',
                                              'Doctorado'], ordered=True) )
    
postulantes['estado'] = ( postulantes['estado'].cat.rename_categories(
                                                {'Abandonado':'abandonado',
                                                 'En Curso':'cursando',
                                                 'Graduado':'graduado'})
                                             .cat.reorder_categories([
                                                  'abandonado',
                                                  'cursando',
                                                  'graduado'], ordered=True) )
educacion = postulantes[['nivel_educacion','estado']].dropna().copy() 



sns.countplot(y='nivel_educacion',hue='estado',data=educacion,
             order=educacion['nivel_educacion'].cat.categories)
# nivel_educacion.plot(kind = 'barh', legend=True);

plt.xlabel("Cantidad", fontsize=14)
plt.ylabel("Nivel de Educación", fontsize=14)
plt.title("Proporción según Nivel de Educación", fontsize=18)
plt.tight_layout()

Pero hay que tener cuidado con estos resultados, porque hay postulantes repetidos.

In [0]:
def reset_index(df):
  "Devuelve un DataFrame con el indice como columnas"
  '''
  Parameters
  ----------
  df DataFrame
  
  Returns
  ------
  '''
  index_df = df.index.to_frame(index=False)
  df = df.reset_index(drop=True)
  
  return pd.merge(index_df,df,left_index=True,right_index=True)

In [0]:
# postulantes.duplicated(subset=['id_postulante','nivel_educacion','estado']).sum()
estado_educacion = postulantes[['id_postulante',
                                'nivel_educacion',
                                'estado']].dropna()
estado_educacion['dummy'] = 1
estado_educacion = (pd.pivot_table(estado_educacion,
                                   index=['id_postulante','nivel_educacion'],
                                   columns='estado')['dummy'].notnull())
                                 
estado_educacion =  estado_educacion.query('abandonado or cursando or graduado')

Para cada postulante elegimos el mayor nivel educativo del que se haya graduado

In [0]:
#: Postulantes que abandonaron estan cursando y graduados
postulantes_acg = (estado_educacion.query('abandonado and cursando and graduado')
                           .sort_index())
print(postulantes_acg)
#Verifico que no haya postulantes que tengan estado ACG para mas de un nivel
postulantes_acg.groupby(level=0).sum().query('(abandonado > 1) or (cursando > 1) or (graduado > 1)')

In [0]:
# Revisamos los datos del postulante con más de un nivel_educacion en todos los estados
print(postulantes_acg.query('id_postulante == "YlMLGD"'),'\n')
print(postulantes.query('id_postulante == "YlMLGD"')
                 .sort_values(by=['nivel_educacion','estado']))

In [0]:
# Generamos 
postulantes_ac = (estado_educacion.query('abandonado and cursando and not graduado')
                           .sort_index())

postulantes_cg = (estado_educacion.query('cursando and graduado and not abandonado')
                           .sort_index())

postulantes_ag = (estado_educacion.query('abandonado and graduado and not cursando')
                           .sort_index())

print(postulantes_ac,
      postulantes_ag,
      postulantes_cg,
      sep='\n')

In [0]:
def filtrar_por_estado_nivel_educativo(postulantes):
    return (postulantes.groupby('id_postulante')
                       .agg('count') 
                       .query('abandonado > 1'))

cuentas_ac = filtrar_por_estado_nivel_educativo(postulantes_ac)
cuentas_ag = filtrar_por_estado_nivel_educativo(postulantes_ag)
cuentas_cg = filtrar_por_estado_nivel_educativo(postulantes_cg)

    
print(postulantes.query('id_postulante in @cuentas_ac.index')
                 .sort_values(by=['id_postulante','nivel_educacion','estado']),'\n')
print(postulantes.query('id_postulante in @cuentas_ag.index')
                 .sort_values(by=['id_postulante','nivel_educacion','estado']),'\n')
print(postulantes.query('id_postulante in @cuentas_cg.index')
                 .sort_values(by=['id_postulante','nivel_educacion','estado']))

Acá hay que filtrar los resultados sobre los que tenemos un conocimiento determinista sobre la educación.
Por ejemplo alguien que tenga un estado: 

nivel_educacion | estado
--: | --:
Master | Abandonado
Master | Cursando

Cómo decidimos si la primera vez que se registró abandonó los estudios y después se registró cuando retomó. O si primero estaba cursando y luego abandonó.

Por lo que tenemos que definir un criterio de selección: 
- Primero tomamos los graduados y elegimos el mayor nivel de educacion.

`multiple_nivel.query('graduado')
              .groupby(level=0,as_index=False)
              .nth(-1)`

el *groupby(level=0)* agrupa por el primer nivel del multiindice 

- Luego tomamos los que estan cursando, no abandonaron y no estan graduados. Y seleccionamos el mayor nivel de educacion.


`multiple_nivel.query('cursando 
                      and not abandonado 
                      and not graduado')
              .groupby(level=0,as_index=False)
              .nth(-1)
`

- Selececcionamos los postulantes restantes, y seleccionamos el anteultimo nivel de educacion.

In [0]:
#Primero vemos los que solo tienen un unico nivel de educacion
un_solo_nivel = estado_educacion.groupby(level=0).count().query('abandonado == 1')
cantidad_postulantes = estado_educacion.groupby(level=0).count().shape


estado_un_solo_nivel = estado_educacion.query('id_postulante in @un_solo_nivel.index')
multiple_nivel = estado_educacion.query('id_postulante not in @un_solo_nivel.index')

print('Hay {:d} filas de postulantes'.format(estado_educacion.shape[0]))
print('Hay {:d} filas de postulantes con un unico nivel educativo'.format(estado_un_solo_nivel.shape[0]))
print('Hay {:d} filas de postulantes con varios niveles'.format(multiple_nivel.shape[0]))



graduados = ( multiple_nivel.query('graduado')
                            .groupby(level=0,as_index=False)
                            .nth(-1) )

cursando =  ( multiple_nivel.query('id_postulante not in @graduados.index \
                                   and ( cursando and not abandonado and not graduado )')
                            .groupby(level=0,as_index=False)
                            .nth(-1) )

# Seleccionamos el anteultimo nivel educativo de los postulantes restantes
restantes = ( multiple_nivel.query('id_postulante not in @graduados.index \
                                    and id_postulante not in @cursando.index')
                            .groupby(level=0,as_index=False)
                            .nth(-2) )

tamanios = tuple(set_.shape[0] for set_ in [graduados,cursando,restantes])

print('Hay {tamanios[0]} filas de graduados,\n\
    {tamanios[1]} filas de postulantes cursando,\n\
    {tamanios[2]} filas restantes.\n'.format(tamanios=tamanios))
print('Lo que da un total de {} filas'.format(sum(tamanios)))

ultimo_estado_educacion = pd.concat([estado_un_solo_nivel,
                                    cursando,graduados,restantes],axis=0)


# print(pd.merge(graduados,cursando,how='inner',on=None))

unico_estado_educacion = ( ultimo_estado_educacion.stack(level=-1)
                               .reset_index()
                               .rename(columns={0:'values'})
                               .query('values')
                               .drop(columns='values'))

print('El dataset final tiene un total de {}\
filas de postulantes unicos'.format(unico_estado_educacion.shape[0]))

In [0]:
'0zB026d' in graduados.index

In [0]:
plt.subplot(2,1,1)
ax1 = sns.countplot(y='nivel_educacion',hue='estado',data=educacion,
             order=educacion['nivel_educacion'].cat.categories)
# nivel_educacion.plot(kind = 'barh', legend=True);

plt.xlabel("Cantidad", fontsize=14)
plt.ylabel("Nivel de Educación", fontsize=14)
plt.title("Proporción según Nivel de Educación (datos crudos)", fontsize=18)
plt.tight_layout()
ax1.set_xscale('log')

plt.subplot(2,1,2)
ax2 = sns.countplot(y='nivel_educacion',hue='estado',data=unico_estado_educacion,
             order=unico_estado_educacion['nivel_educacion'].cat.categories)
# nivel_educacion.plot(kind = 'barh', legend=True);

plt.xlabel("Cantidad", fontsize=14)
plt.ylabel("Nivel de Educación", fontsize=14)
plt.title("Proporción según Nivel de Educación (datos filtrados)", fontsize=18)
plt.tight_layout() 
ax2.set_xscale('log')

ax_lim = max(ax1.get_xlim()[1],ax1.get_xlim()[1])
ax1.set_xlim((0,ax_lim))
ax2.set_xlim((0,ax_lim))

In [0]:
# unico_estado_educacion.groupby('id_postulante').count()
fpp = (ultimo_estado_educacion.stack(level=-1)
                               .reset_index()
                               .rename(columns={0:'values'}) 
                               .query('values'))

fpp.groupby('id_postulante').count()

### 2.3 Sexo

In [0]:
sexo_postulantes = postulantes[['id_postulante','sexo']].drop_duplicates()
# Verificamos de que no exista postulante multiples declaraciones de sexo
sexo_postulantes['id_postulante'].value_counts(sort=True,ascending=False)

# Graficamos la cantidad de postulantes por sexo

sns.countplot(x='sexo',data=sexo_postulantes)
print(sexo_postulantes['sexo'].value_counts())

In [0]:
# Porcentaje de sexo de los postulantes

porcentaje_sexo_postulantes = sexo_postulantes.groupby(sexo_postulantes.sexo, as_index = False).size().reset_index(name='suma')

porcentaje_sexo_postulantes['porcentaje'] = (porcentaje_sexo_postulantes['suma'] * 100.)/(porcentaje_sexo_postulantes['suma'].sum())

sns.set_style("whitegrid")
sns.barplot(x='sexo', y='porcentaje', data=porcentaje_sexo_postulantes, color = "SteelBlue")
plt.xlabel("Sexo", fontsize=14)
plt.ylabel("Porcentaje", fontsize=14)
plt.title("Porcentaje de postulantes según sexo", fontsize=18)
plt.tight_layout()

### 2.3 Edad vs sexo

In [0]:
anio_nacimiento_sexo = (postulantes[['fecha_nacimiento','sexo']]
                                  .query('fecha_nacimiento > "1900-01-01" ')
                                  .copy())
anio_nacimiento_sexo['anio_nacimiento'] = ( anio_nacimiento_sexo['fecha_nacimiento']
                                             .dt.year )

#   plt.subplot(1,2,1)
sns.countplot(x='anio_nacimiento',hue='sexo',data=anio_nacimiento_sexo
                                                  .query('anio_nacimiento <= 1970'))
plt.xticks(rotation=90)
#   plt.subplot(1,2,2)
plt.show()
sns.countplot(x='anio_nacimiento',hue='sexo',data=anio_nacimiento_sexo
                                                  .query('anio_nacimiento > 1970'))
plt.xticks(rotation=90)

plt.tight_layout()

**CONCLUSIÓN** El comportamiento Normal de asimetriza Izquierda.

**OBSERVACION** Vemos que la cantidad de "no_declara"es muy baja respecto a Femenino y masculino. Para ver en mejor detalle sobre como se comportan, analizamos aparte la cantidad de "no_declara""



In [0]:
sns.countplot(x='anio_nacimiento',data=anio_nacimiento_sexo.query('sexo == "NO_DECLARA"'),\
             color='Black')
plt.xticks(rotation=90)

**CONCLUSIÓN** El comportamiento Normal de asimetriza Izquierda.


### 2.4 Edad vs educacion

In [0]:
anio_educacion = postulantes[]

### Educacion vs sexo

In [0]:
postulantes_ed_sexo = postulantes[['id_postulante','sexo','nivel_educacion']].groupby(['sexo','nivel_educacion'],as_index=False).count()
ax = sns.barplot(x='nivel_educacion',y='id_postulante',hue='sexo',data=postulantes_ed_sexo,
                order=educacion['nivel_educacion'].cat.categories)
# ax = sns.barplot(x="day", y="total_bill", hue="sex", data=tips)
plt.tight_layout()

**CONCLUSION** Las personas que mas aplican son las que menor educacion tienen. Entonces nos preguntamos: **es importante un master, doctorado o posgrado para asegurar el trabajo?** 

Buscaremos responderlo a continuación.

In [0]:
a = postulantes['nivel_educacion'].value_counts()

#df.reset_index(level=['tick', 'obs'])

#a.reset_index(level=a.index.names, inplace=True)
a.reindex(a)
a


In [0]:
a.describe

## Puede un postulante visitar una postulacion despues de haberse postulado?

### CONCLUSION: 
1. Notamos una gran concentración en Secundario y Universitario 

### Buscamos: Que numero de postulantes estan tanto en las visitas como en las postulaciones


In [0]:
postulaciones['id_postulante'].isin(visitas['id_postulante']).value_counts()

# es necesario visitar el articulo para postularme?

### Agrupamos id_postulante por id_aviso


In [0]:
(postulaciones.groupby(['id_aviso','id_postulante'])
              .count()
              .sort_values(by='fecha_postulacion',ascending=False))

### Buscamos saber si hay multiples visitas por id_postulante y fecha_visita

In [0]:
visitas_multiples = ( visitas.groupby(['id_aviso','id_postulante'], as_index = False)
                            .count()
                            .sort_values(by='fecha_visita',ascending=False) )
visitas_multiples = visitas_multiples[visitas_multiples['fecha_visita'] > 1]
# def diff_time(df):
#     return df.max() - df.min()
visitas_fechas = ( visitas.merge(visitas_multiples, 
                               left_on=['id_aviso','id_postulante'],
                               right_on=['id_aviso','id_postulante'],
                               how='inner') )
visitas_fechas = visitas_fechas.drop(columns='fecha_visita_y')
visitas_fechas.groupby(by=['id_aviso','id_postulante']).agg(['min','max'])

In [0]:
# visitas_multiples
# visitas_fechas.sort_values(by=['id_aviso','id_postulante'])
# vistas_fechas.groupby(by=['id_aviso','id_postulante']).count()
visitas_fechas.count()

In [0]:
# with pd.option_context('display.max_rows', 20):
visitas[(visitas['id_aviso'] == 1112334789) &	(visitas['id_postulante'] == 'VNeZ482')].sort_values(by='fecha_visita')

Acá vemos que hay varios usuarios que visitan el mismo aviso varias veces, por lo que decidimos ver el período de tiempo en que se vió el anuncio, y en este caso, el postulante vió 116 veces el anuncio en 1 minuto

In [0]:
with pd.option_context('display.max_rows', 20):
  postulaciones[(postulaciones["id_aviso"] == 1112349911)  & (postulaciones["id_postulante"] == "NzrBbLz")]

**CONCLUSION** Vemos que hay una faltante importante en columnas como *ciudad*

In [0]:
# visitas.query('idAviso in @id')
# visitas
print('Rango de visitas:\n')
print(visitas['fecha_visita'].agg(['min','max']),'\n')

print('Rango de postulaciones:\n')
print(postulaciones['fecha_postulacion'].agg(['min','max']))

## 1.3 Estudiamos si hay valores Nulos en dataframes
Nos ayuda a normalizar los datasets y purificar la informacion.

In [0]:
!pip install -q altair
from altair import Chart, X, Y, Axis, SortField

In [0]:
Chart(df)

## REPORTE Global avisos_Detalle
Queremos conocer el comprotamiento de la tabla AVISOS_DETALLE

In [0]:
!pip install pandas_profiling
import pandas_profiling
pandas_profiling.ProfileReport(avisos_detalle);

In [0]:
avisos_detalle.head()

In [0]:

g = sns.countplot(x='zona_de_trabajo',data=avisos_detalle, hue='tipo_de_trabajo')  #hue='nivel_laboral?
plt.xticks(rotation=30)


In [0]:
print (avisos_detalle['tipo_de_trabajo'].value_counts())

# ANALISIS           PONER *TITULO*

## GRAFICO 3.1: Grafico de cantidad de postulantes por género


In [0]:
# Grafico de cantidad de postulantes por género
postulantes_sexo = (postulantes['sexo'].value_counts()/postulantes['sexo'].count())*100
postulantes_sexo.plot(kind = 'barh', legend=True, color = "Steelblue");

plt.xlabel("Porcentaje", fontsize=14)
plt.ylabel("Sexo", fontsize=14)
plt.title("% Según sexo ", fontsize=18)
plt.legend()

plt.tight_layout()

## GRAFICO 3.1: Histograma de edad de los postulantes

In [0]:
import matplotlib.mlab as mlab


# Histograma de edad de los postulantes

postulantes['ano_nacimiento'] = postulantes['fecha_nacimiento'].dt.year
postulantes['edad'] = (2018 - postulantes['ano_nacimiento'])

postulantes_edad = postulantes
postulantes_edad = postulantes_edad[postulantes_edad.edad < 80]
postulantes_edad['edad'].hist(bins = 30, );
plt.tight_layout()

#Estadisticos 
media =postulantes['edad'].mean()
desvio= postulantes['edad'].std()


# add a 'best fit' line
y = mlab.normpdf( 30, media, desvio)
l = plt.plot(30, y, 'r-', linewidth=1)

In [0]:
postulantes

## 3.4 GRAFICO : Grafico de Barras agrupando por nivel de Educacion y Sexo vs Cantidad postulantes.



# es importante un master, doctorado o posgrado para asegurar el trabajo?
Estudio la correlacion entre variables. Para aplicar corr() debo asegurar que las variables sean del tipo numerico

In [0]:
postulantes.head()

In [0]:
#typo de data

postulantes.dtypes

Como ninguno es del tipo numerico debemos buscar la forma de pasar a un analisis numerico. Para ello buscamos conocer la cantidad de NIVEL_EDUCACION por NOMBRA_AREA_LABORAL:

In [0]:
print (visitas.head())

# TIME SPLIT

## TIME Split : en TABLA_VISITA
Nos dimos cuenta que el formato de de la columna "fecha_visita"no nos es comodo para trabajar. Para acomodarlo, buscamos separar la hora ("Time") y el dia ("Date")

In [0]:
#Separamos Date y Time
temp = pd.DatetimeIndex(visitas['fecha_visita'])
visitas['fecha_visita'] = temp.date
visitas['horario_visita'] = temp.time

visitas.head()





In [0]:
# Acomodamos el formato
temp = pd.DatetimeIndex(postulaciones['fecha_postulacion'])
postulaciones['fecha_postulacion'] = temp.date
postulaciones['horario_postulacion'] = temp.time

postulaciones.head()

### *Cuales* son los avisos mas vistos?
Buscamos estudiar cules son los Avisos mas vistos (Sin contar las repeticiones que ocurrieron en un mismo día) Esto es importante dado que nos interesa conocer la PRESENCIA DIARIA DEL USUARIO.

In [0]:
#df[['col1', 'col2', 'col3', 'col4']].groupby(['col1', 'col2']).agg(['mean', 'count'])
#visitas[['idpostulante','idAviso']].groupby(['idAviso']).agg(['count'])
#grouped = visitas.groupby('idAviso').count().reset_index()
#grouped.sort_values('timestamp', ascending=False)

aux2 = visitas[['id_aviso','id_postulante','Date']].drop_duplicates()
aux = aux2['id_aviso']
aux.value_counts().head()



## Cuales son los avisos mas vistos?


In [0]:
aux = aux2['id_postulante']
aux.value_counts().head()

## que aviso se postula mas la gente? avisos y postulaciones

In [0]:

aux2 = postulaciones[['id_aviso','id_postulante','Date']].drop_duplicates()
aux = aux2['id_aviso']
aux.value_counts().head()

## Que postulantes se postulan mas veces? cantidad por idpostulante

In [0]:
aux = aux2['id_postulante']
aux.value_counts().head()

## Estudiamos los periodos de visita
Para ello, analizamos maximo y minimo de fechas segun id_aviso

In [0]:
def dadoIDavisoDevolverPeriodoVisto(idAviso):
  return visitas[visitas['id_aviso'] == idAviso]['Date'].agg(['min','max'])

In [0]:
dadoIDavisoDevolverPeriodoVisto(1112033906)

In [0]:

3.
diferencia en sexo por nombre_area
diferencia en edad por nombre_area
4.
diferencia en sexo por tipo_de_trabajo
diferencia en edad por tipo_de_trabajo
5.
diferencia en sexo por nivel_laboral
diferencia en edad por nivel_laboral

diferencia en sexo por nombre_area


# titulo??????????


## CREAMOS LA TABLA Postulaciones_genero_edad_postulaciones_con_detalle : merge dePOSTULANTES_GENERO, POSTULACIONES y AVISOS_DETALLES


In [0]:
#Armamos el merge

postulantes_genero_edad_postulaciones  =  pd.merge( postulantes_genero_edad, postulaciones,  how = 'inner' ,  on = [ 'id_postulante'])
postulantes_genero_edad_postulaciones_con_detalle  =  pd.merge( postulantes_genero_edad_postulaciones, avisos_detalle, how = 'inner' ,  on = [ 'id_aviso'])

#TIME Split

#Separamos Date y Time
temp2 = pd.DatetimeIndex(postulantes_genero_edad_postulaciones_con_detalle['fecha_postulacion'])
postulantes_genero_edad_postulaciones_con_detalle['Date'] = temp2.date
postulantes_genero_edad_postulaciones_con_detalle['Time'] = temp2.time

#Generamos una columna con los DIAS DE Date
postulantes_genero_edad_postulaciones_con_detalle['Date'] = pd.to_datetime(postulantes_genero_edad_postulaciones_con_detalle['Date'])
postulantes_genero_edad_postulaciones_con_detalle['day_of_week'] = postulantes_genero_edad_postulaciones_con_detalle['Date'].dt.weekday_name

#Agregamos una columna EDAD
postulantes_genero_edad_postulaciones_con_detalle['edad']=postulantes_edad["edad"]

#Imprimo
postulantes_genero_edad_postulaciones_con_detalle.head(1)



In [0]:
ax = sns.swarmplot(x="day_of_week", y=postulantes_genero_edad_postulaciones_con_detalle['nombre_area_laboral'].value_counts(), data=postulantes_genero_edad_postulaciones_con_detalle)
#ax = sns.boxplot(x="day_of_week",
                 data=postulantes_genero_edad_postulaciones_con_detalle,
        showcaps=False,boxprops={'facecolor':'None'},
        showfliers=False,whiskerprops={'linewidth':0})

plt.show()


Fijate de definir un nuevo dataframe que tenga solo las columnas que te interesan


In [0]:
df = postulantes_genero_edad_postulaciones_con_detalle[['nombre_area_laboral',
                                                             'day_of_week']]
df['dummy'] = 1 #agrego columna para poder agrupar
df = df.groupby(by=['day_of_week','nombre_area_laboral'],as_index=False).count()

df2 = df.sample(n=500)
ploti = sns.swarmplot(x='day_of_week',y='dummy',hue='nombre_area_laboral', data=df2)




In [0]:
#Reviso el dtype de la columna "day_of_week"
print(postulantes_genero_edad_postulaciones_con_detalle.dtypes)

Veo que "day_of_week"es **object** busco ahora, que sea **categorical** para trabajar con Swarmplot ;) en el proximo punto.

In [0]:
#Convierto "day_of_week"de *objeto* a *category*
postulantes_genero_edad_postulaciones_con_detalle["day_of_week"] = postulantes_genero_edad_postulaciones_con_detalle["day_of_week"].astype('category')

#verifico conversion
print(postulantes_genero_edad_postulaciones_con_detalle.dtypes)


## Contar la cantidad de postulaciones a nombre_area_Laboral por day_of _week
Buscamos saber la cantidad para luego representarlo y asi entender el comportamiento de las nombre_areas_laboral durante los dias de la semana.

In [0]:
#postulantes_genero_edad_postulaciones_con_detalle['nombre_area_laboral'].value_counts
postulantes_agrupados_1 = postulantes_genero_edad_postulaciones_con_detalle.groupby(['nombre_area_laboral'],as_index=True).count()["id_postulante"].sort_values(ascending=False)
postulantes_agrupados_1.head(20)


In [0]:
ax = sns.swarmplot(x="day_of_week", y="total_bill",hue=nombre_area_laboral, data=postulantes_genero_edad_postulaciones_con_detalle)
#ax = sns.boxplot(x="day", y="total_bill", data=tips,
        #showcaps=False,boxprops={'facecolor':'None'},
        #showfliers=False,whiskerprops={'linewidth':0})

plt.show()

# JUEGO GRAFICO raros


## ANALISIS ESTADISTICO

1) corr() toda variables numericas deben ser
https://github.com/afolaborn/Python_Jupyter_Notebook/blob/master/Correlation/Correlation_Matrix_Upload.ipynb

In [0]:
postulantes_genero_edad_postulaciones_con_detalle.head(1)




In [0]:
df = postulantes_genero_edad_postulaciones_con_detalle[['nombre_area_laboral',
                                                             'day_of_week']]
df.head(5)

In [0]:
- EDAD PROMEDIO POR AREA DE TRABAJO . HEATMAP : AREA TRABAJO VS INTERVALOS DE EDADES
  

In [0]:
df2 = 

# Bloques de ZonaJobs
## Educacion en el CV

In [0]:
%%html
<head>
  <link rel="stylesheet" media="all" href="https://imgzj.jobscdn.com/postulantes-assets/zonajobs_bundle_desktop-cf37007141e1bfedc5d639d92eac9ee5.css">
</head>
<body>
<div class="modal-dialog" role="document">
<div class="modal-content">
<div class="modal-header">
<button type="button" class="close" data-dismiss="modal" aria-label="Close"><span aria-hidden="true">×</span></button>
<h3>Educación</h3>
</div>
<div class="modal-body">
<input id="estudioEditId_" type="hidden">				        	
<div class="cv-form">
<form>
<div class="col-md-12 pd0 mb15">
<div class="col-md-6">
<label for="paisEducacion" class="required">País</label>
<div class="select-wrap">
<select id="paisEducacion" name="paisEducacion" style="opacity: 1;">
<option selected="" disabled="">Seleccionar</option>
<option id="34">Alemania</option>
<option id="1">Argentina</option>
<option id="30">Australia</option>
<option id="31">Austria</option>
<option id="5">Bolivia</option>
<option id="2">Brasil</option>
<option id="54">Bulgaria</option>
<option id="53">Bélgica</option>
<option id="23">Canada</option>
<option id="1007">Chile</option>
<option id="52">China</option>
<option id="8">Colombia</option>
<option id="68">Corea</option>
<option id="14">Costa Rica</option>
<option id="56">Croacia</option>
<option id="29">Cuba</option>
<option id="33">Dinamarca</option>
<option id="9">Ecuador</option>
<option id="15">El Salvador</option>
<option id="75">Emiratos Arabes</option>
<option id="71">Escocia</option>
<option id="57">Eslovaquia</option>
<option id="58">Eslovenia</option>
<option id="21">Espana</option>
<option id="24">Estados Unidos</option>
<option id="59">Estonia</option>
<option id="60">Finlandia</option>
<option id="25">Francia</option>
<option id="35">Grecia</option>
<option id="16">Guatemala</option>
<option id="370">Haiti</option>
<option id="38">Holanda</option>
<option id="17">Honduras</option>
<option id="61">Hungria</option>
<option id="62">India</option>
<option id="32">Inglaterra</option>
<option id="1000">Internacional</option>
<option id="74">Irak</option>
<option id="63">Irlanda</option>
<option id="40">Israel</option>
<option id="26">Italia</option>
<option id="39">Japon</option>
<option id="64">Letonia</option>
<option id="65">Lituania</option>
<option id="380">Malasia</option>
<option id="18">Mexico</option>
<option id="19">Nicaragua</option>
<option id="66">Noruega</option>
<option id="67">Nueva Zelanda</option>
<option id="20">Panama</option>
<option id="10">Paraguay</option>
<option id="11">Peru</option>
<option id="50">Polonia</option>
<option id="22">Portugal</option>
<option id="27">Puerto Rico</option>
<option id="200">Republica Checa</option>
<option id="28">Republica Dominicana</option>
<option id="1001">República de Malta</option>
<option id="69">Rumania</option>
<option id="37">Rusia</option>
<option id="70">Singapur</option>
<option id="73">Suecia</option>
<option id="36">Suiza</option>
<option id="12">Uruguay</option>
<option id="13">Venezuela</option>
</select>
<div class="select-arrow"></div>
</div><!-- select-wrap -->
</div>
</div>
<div class="col-md-12 pd0 mb15">
<div class="col-md-6">
<label for="tipoEstudio" class="required">Tipo de estudio</label>
<div class="select-wrap">
<select id="tipoEstudio" class="form-control" style="opacity: 1;">
<option disabled="" selected="">Seleccionar</option>
<option id="1">Secundario</option>
<option id="2">Terciario</option>
<option id="3">Universitario</option>
<option id="4">Posgrado</option>
<option id="5">Master</option>
<option id="6">Doctorado</option>
<option id="7">Otro</option>
</select>
<div class="select-arrow"></div>
</div><!-- select-wrap -->
</div>
<div class="col-md-6">
<label for="estadoEstudio" class="required">Estado</label>
<div class="select-wrap">
<select id="estadoEstudio" class="form-control" style="opacity: 1;">
<option disabled="" selected="">Seleccionar</option>
<option id="1">En Curso</option>
<option id="2">Graduado</option>
<option id="3">Abandonado</option>
</select>
<div class="select-arrow"></div>
</div><!-- select-wrap -->
</div>					        
</div>
<div class="col-md-12 pd0 mb15">
<div class="col-md-6">
<label for="titulo" class="required">Título / Carrera</label>
<input id="titulo" name="titulo" type="text" style="opacity: 1;">
</div>
<div class="col-md-6">
<label for="nacionalidad" class="required">Área de estudio</label>
<div class="select-wrap">				                 
<select id="areaEstudio" class="form-control" style="opacity: 1;">
<option disabled="" selected="">Seleccionar</option>
<option id="1">Abogacía / Derecho / Leyes</option>
<option id="233">Actuaría</option>
<option id="57">Acuicultura</option>
<option id="2">Adm. de Empresas</option>
<option id="58">Adm. y Gestión Pública</option>
<option id="278">Administración Agropecuaria</option>
<option id="250">Agrimensor</option>
<option id="60">Agronegocios</option>
<option id="62">Antropología</option>
<option id="61">Análisis de Sistemas</option>
<option id="266">Apoderado Aduanal</option>
<option id="63">Arqueología</option>
<option id="4">Arquitectura</option>
<option id="267">Asesoría Legal Internacional</option>
<option id="274">Asesoría en Comercio Exterior</option>
<option id="273">Asesoría en Comercio Exterior Jubilado</option>
<option id="268">Asistente de Tráfico</option>
<option id="64">Astronomía</option>
<option id="193">Bachiller</option>
<option id="406">Bacteriología</option>
<option id="6">Bellas Artes</option>
<option id="88">Bibliotecología</option>
<option id="234">BioFisica</option>
<option id="252">Bioingeniería</option>
<option id="7">Biología</option>
<option id="89">Bioquímica</option>
<option id="238">Biotecnología</option>
<option id="280">Call Center</option>
<option id="269">Capacitación Comercio Exterior</option>
<option id="66">Cartografía</option>
<option id="9">Ciencias Físicas</option>
<option id="8">Ciencias Políticas</option>
<option id="265">Ciencias de la Educación</option>
<option id="232">Ciencias del Ejercicio / Educacion Física</option>
<option id="67">Comercio Int./Ext.</option>
<option id="270">Compras Internacionales/Importación</option>
<option id="10">Computación / Informática</option>
<option id="68">Comunicación Audiovisual</option>
<option id="41">Comunicación Social</option>
<option id="12">Construcción / Obras Civiles</option>
<option id="271">Consultorías Comercio Exterior</option>
<option id="11">Contabilidad / Auditoría</option>
<option id="70">Dibujo Técnico</option>
<option id="13">Diseño Gráfico</option>
<option id="71">Diseño Industrial</option>
<option id="402">Diseño Web</option>
<option id="231">Diseño de Imagen y Sonido</option>
<option id="72">Diseño de Vestuario / Textil / Modas</option>
<option id="73">Ecología</option>
<option id="14">Economía</option>
<option id="194">Educacion</option>
<option id="128">Electricidad</option>
<option id="91">Electrónica</option>
<option id="15">Enfermería</option>
<option id="407">Enfermería</option>
<option id="401">Enología</option>
<option id="255">Escribanía</option>
<option id="131">Estadística</option>
<option id="132">Farmacia</option>
<option id="408">Farmacia</option>
<option id="18">Filosofía</option>
<option id="134">Finanzas</option>
<option id="272">Finanzas Internacionales</option>
<option id="19">Fisioterapia</option>
<option id="409">Fisioterapia</option>
<option id="220">Fonoaudiologia</option>
<option id="410">Fonoaudiología</option>
<option id="20">Fotografía</option>
<option id="137">Gastronomía / Cocina</option>
<option id="257">Geofísica</option>
<option id="23">Geografía</option>
<option id="139">Geología / Geomensura / Topografía</option>
<option id="140">Hidráulica</option>
<option id="411">Histopatología</option>
<option id="22">Historia</option>
<option id="24">Hotelería</option>
<option id="236">Ing. - otros</option>
<option id="25">Ing. Aerospacial</option>
<option id="279">Ing. Agropecuario</option>
<option id="200">Ing. Agrónomo</option>
<option id="77">Ing. Alimentos</option>
<option id="52">Ing. Ambiental</option>
<option id="3">Ing. Comercial</option>
<option id="28">Ing. Electrónica</option>
<option id="27">Ing. Eléctrica</option>
<option id="78">Ing. Forestal</option>
<option id="29">Ing. Hidraúlica</option>
<option id="26">Ing. Industrial</option>
<option id="30">Ing. Informática</option>
<option id="53">Ing. Matemática</option>
<option id="54">Ing. Mecánica/Metalúrgica</option>
<option id="31">Ing. Naval</option>
<option id="259">Ing. Nuclear</option>
<option id="32">Ing. Obras Civiles/Construcción</option>
<option id="79">Ing. Pesquera / Cultivos Marinos</option>
<option id="160">Ing. Petróleo</option>
<option id="33">Ing. Química</option>
<option id="251">Ing. Recursos Hídricos</option>
<option id="34">Ing. Sonido</option>
<option id="92">Ing. Telecomunicaciones</option>
<option id="35">Ing. Transporte</option>
<option id="210">Ing. en Materiales</option>
<option id="55">Ing. en Minas</option>
<option id="310">Ing. en Sistemas</option>
<option id="400">Ingeniero en construcción</option>
<option id="390">Ingeniero vial</option>
<option id="264">Intérprete</option>
<option id="412">Kinesiología</option>
<option id="254">Kinesiología</option>
<option id="93">Laboratorio (Mecánica) Dental</option>
<option id="80">Literatura</option>
<option id="195">Maestro Mayor de Obras</option>
<option id="163">Marketing / Comercialización</option>
<option id="37">Matemáticas</option>
<option id="94">Mecánica / Metalúrgica</option>
<option id="36">Medicina</option>
<option id="413">Medicina</option>
<option id="167">Medio Ambiente</option>
<option id="275">Mercadotecnia Internacional</option>
<option id="414">Microbiología</option>
<option id="168">Minería / Petróleo / Gas</option>
<option id="38">Música</option>
<option id="82">Nutrición</option>
<option id="415">Nutrición</option>
<option id="83">Oceanografía</option>
<option id="39">Odontología</option>
<option id="416">Odontología</option>
<option id="256">Organización Industrial</option>
<option id="235">Otra</option>
<option id="85">Paisajismo</option>
<option id="40">Periodismo</option>
<option id="196">Perito Mercantil</option>
<option id="237">Procesos / Calidad Total</option>
<option id="86">Programación</option>
<option id="403">Psicología</option>
<option id="175">Psicología</option>
<option id="404">Psicopedagogía</option>
<option id="176">Psicopedagogía</option>
<option id="42">Publicidad</option>
<option id="43">Química</option>
<option id="417">Químico Farmacéutico</option>
<option id="179">Recursos Humanos / Relac. Ind.</option>
<option id="96">Relaciones Internac.</option>
<option id="44">Relaciones Públicas</option>
<option id="405">Salud</option>
<option id="45">Secretariado</option>
<option id="97">Seguridad Industrial</option>
<option id="261">Seguros</option>
<option id="46">Sociología</option>
<option id="197">Tecnico</option>
<option id="185">Tecnología Médica / Laboratorio</option>
<option id="262">Tecnología de Alimentos</option>
<option id="276">Tecnologías de la Información</option>
<option id="186">Telecomunicaciones</option>
<option id="99">Terapia Ocupacional</option>
<option id="418">Terapia Ocupacional</option>
<option id="48">Trabajo Social</option>
<option id="189">Traducción </option>
<option id="190">Transporte</option>
<option id="50">Turismo</option>
<option id="277">Ventas Internacionales/Exportación</option>
<option id="51">Veterinaria</option>
<option id="419">Veterinaria</option>
</select>
<div class="select-arrow"></div>
</div><!-- select-wrap -->
</div>					        
</div>
<div class="col-md-12 pd0 mb15">
<div class="col-md-6 fnac" id="fechaInicioEstudioWrap">
<label for="fechaInicioEstudio" class="required">Fecha de inicio</label>
<div class="row">
<div class="col-md-6">
<div class="select-wrap">
<select id="mesInicioEstudio" name="mesInicioEstudio" style="opacity: 1;">
<option id="" value="" selected="" disabled="">Mes</option>
<option id="ie01" value="01">Ene</option>
<option id="ie02" value="02">Feb</option>
<option id="ie03" value="03">Mar</option>
<option id="ie04" value="04">Abr</option>
<option id="ie05" value="05">May</option>
<option id="ie06" value="06">Jun</option>
<option id="ie07" value="07">Jul</option>
<option id="ie08" value="08">Ago</option>
<option id="ie09" value="09">Sep</option>
<option id="ie10" value="10">Oct</option>
<option id="ie11" value="11">Nov</option>
<option id="ie12" value="12">Dic</option>
</select>
<div class="select-arrow"></div>
</div><!-- select-wrap -->
</div><!-- col -->
<div class="col-md-6">
<div class="select-wrap">
<select id="anioInicioEstudio" name="anioInicioEstudio" style="opacity: 1;">
<option id="" value="" selected="" disabled="">Año</option>
<option id="2018" value="2018">2018</option><option id="2017" value="2017">2017</option><option id="2016" value="2016">2016</option><option id="2015" value="2015">2015</option><option id="2014" value="2014">2014</option><option id="2013" value="2013">2013</option><option id="2012" value="2012">2012</option><option id="2011" value="2011">2011</option><option id="2010" value="2010">2010</option><option id="2009" value="2009">2009</option><option id="2008" value="2008">2008</option><option id="2007" value="2007">2007</option><option id="2006" value="2006">2006</option><option id="2005" value="2005">2005</option><option id="2004" value="2004">2004</option><option id="2003" value="2003">2003</option><option id="2002" value="2002">2002</option><option id="2001" value="2001">2001</option><option id="2000" value="2000">2000</option><option id="1999" value="1999">1999</option><option id="1998" value="1998">1998</option><option id="1997" value="1997">1997</option><option id="1996" value="1996">1996</option><option id="1995" value="1995">1995</option><option id="1994" value="1994">1994</option><option id="1993" value="1993">1993</option><option id="1992" value="1992">1992</option><option id="1991" value="1991">1991</option><option id="1990" value="1990">1990</option><option id="1989" value="1989">1989</option><option id="1988" value="1988">1988</option><option id="1987" value="1987">1987</option><option id="1986" value="1986">1986</option><option id="1985" value="1985">1985</option><option id="1984" value="1984">1984</option><option id="1983" value="1983">1983</option><option id="1982" value="1982">1982</option><option id="1981" value="1981">1981</option><option id="1980" value="1980">1980</option><option id="1979" value="1979">1979</option><option id="1978" value="1978">1978</option><option id="1977" value="1977">1977</option><option id="1976" value="1976">1976</option><option id="1975" value="1975">1975</option><option id="1974" value="1974">1974</option><option id="1973" value="1973">1973</option><option id="1972" value="1972">1972</option><option id="1971" value="1971">1971</option><option id="1970" value="1970">1970</option><option id="1969" value="1969">1969</option><option id="1968" value="1968">1968</option><option id="1967" value="1967">1967</option><option id="1966" value="1966">1966</option><option id="1965" value="1965">1965</option><option id="1964" value="1964">1964</option><option id="1963" value="1963">1963</option><option id="1962" value="1962">1962</option><option id="1961" value="1961">1961</option><option id="1960" value="1960">1960</option><option id="1959" value="1959">1959</option><option id="1958" value="1958">1958</option><option id="1957" value="1957">1957</option><option id="1956" value="1956">1956</option><option id="1955" value="1955">1955</option><option id="1954" value="1954">1954</option><option id="1953" value="1953">1953</option><option id="1952" value="1952">1952</option><option id="1951" value="1951">1951</option><option id="1950" value="1950">1950</option></select>
<div class="select-arrow"></div>
</div><!-- select-wrap -->
</div><!-- col -->
</div><!-- row -->
<input id="fechaInicioEstudio" name="fechaInicioEstudio" type="hidden" class="calendar">
<!-- <label for="fechaInicioEstudio" class="icon-calendar calendar-trigger"></label> -->
</div>
<div class="col-md-6 form-group fnac" id="fechaFinEstudioWrap">
<label for="fechaFinEstudio" class="required">Fecha de fin</label>
<div class="row">
<div class="col-md-6">
<div class="select-wrap">
<select id="mesFinEstudio" name="mesFinEstudio" style="opacity: 1;">
<option id="" value="" selected="" disabled="">Mes</option>
<option id="fe01" value="01">Ene</option>
<option id="fe02" value="02">Feb</option>
<option id="fe03" value="03">Mar</option>
<option id="fe04" value="04">Abr</option>
<option id="fe05" value="05">May</option>
<option id="fe06" value="06">Jun</option>
<option id="fe07" value="07">Jul</option>
<option id="fe08" value="08">Ago</option>
<option id="fe09" value="09">Sep</option>
<option id="fe10" value="10">Oct</option>
<option id="fe11" value="11">Nov</option>
<option id="fe12" value="12">Dic</option>
</select>
<div class="select-arrow"></div>
</div><!-- select-wrap -->
</div><!-- col -->
<div class="col-md-6">
<div class="select-wrap">
<select id="anioFinEstudio" name="anioFinEstudio" style="opacity: 1;">
<option id="" value="" selected="" disabled="">Año</option>
<option id="2018" value="2018">2018</option><option id="2017" value="2017">2017</option><option id="2016" value="2016">2016</option><option id="2015" value="2015">2015</option><option id="2014" value="2014">2014</option><option id="2013" value="2013">2013</option><option id="2012" value="2012">2012</option><option id="2011" value="2011">2011</option><option id="2010" value="2010">2010</option><option id="2009" value="2009">2009</option><option id="2008" value="2008">2008</option><option id="2007" value="2007">2007</option><option id="2006" value="2006">2006</option><option id="2005" value="2005">2005</option><option id="2004" value="2004">2004</option><option id="2003" value="2003">2003</option><option id="2002" value="2002">2002</option><option id="2001" value="2001">2001</option><option id="2000" value="2000">2000</option><option id="1999" value="1999">1999</option><option id="1998" value="1998">1998</option><option id="1997" value="1997">1997</option><option id="1996" value="1996">1996</option><option id="1995" value="1995">1995</option><option id="1994" value="1994">1994</option><option id="1993" value="1993">1993</option><option id="1992" value="1992">1992</option><option id="1991" value="1991">1991</option><option id="1990" value="1990">1990</option><option id="1989" value="1989">1989</option><option id="1988" value="1988">1988</option><option id="1987" value="1987">1987</option><option id="1986" value="1986">1986</option><option id="1985" value="1985">1985</option><option id="1984" value="1984">1984</option><option id="1983" value="1983">1983</option><option id="1982" value="1982">1982</option><option id="1981" value="1981">1981</option><option id="1980" value="1980">1980</option><option id="1979" value="1979">1979</option><option id="1978" value="1978">1978</option><option id="1977" value="1977">1977</option><option id="1976" value="1976">1976</option><option id="1975" value="1975">1975</option><option id="1974" value="1974">1974</option><option id="1973" value="1973">1973</option><option id="1972" value="1972">1972</option><option id="1971" value="1971">1971</option><option id="1970" value="1970">1970</option><option id="1969" value="1969">1969</option><option id="1968" value="1968">1968</option><option id="1967" value="1967">1967</option><option id="1966" value="1966">1966</option><option id="1965" value="1965">1965</option><option id="1964" value="1964">1964</option><option id="1963" value="1963">1963</option><option id="1962" value="1962">1962</option><option id="1961" value="1961">1961</option><option id="1960" value="1960">1960</option><option id="1959" value="1959">1959</option><option id="1958" value="1958">1958</option><option id="1957" value="1957">1957</option><option id="1956" value="1956">1956</option><option id="1955" value="1955">1955</option><option id="1954" value="1954">1954</option><option id="1953" value="1953">1953</option><option id="1952" value="1952">1952</option><option id="1951" value="1951">1951</option><option id="1950" value="1950">1950</option></select>
<div class="select-arrow"></div>
</div><!-- select-wrap -->
</div><!-- col -->
</div><!-- row -->
<input id="fechaFinEstudio" name="fechaFinEstudio" class="mb5 calendar" type="hidden">
<!-- <label for="fechaFinEstudio" class="icon-calendar calendar-trigger"></label> -->
<div class="checkbox">
<label>
<input id="al-presente-estudio" type="checkbox" selected="" value=""><span class="checkbox-material"><span class="check"></span></span> Al presente
</label>
</div>
</div>
</div>
<div class="col-md-12 pd0 mb15">
<div class="col-md-6 institucion-wrapper">
<label class="control-label required" for="institucionTypeAhead"> Institución</label>
<span class="twitter-typeahead" style="position: relative; display: inline-block;"><input id="institucionTypeAhead" name="institucionTypeAhead" type="text" class="form-control tt-input" autocomplete="off" spellcheck="false" dir="auto" style="position: relative; vertical-align: top; opacity: 1;"><pre aria-hidden="true" style="position: absolute; visibility: hidden; white-space: pre; font-family: Montserrat, Arial, sans-serif; font-size: 14px; font-style: normal; font-variant: normal; font-weight: 400; word-spacing: 0px; letter-spacing: 0px; text-indent: 0px; text-rendering: auto; text-transform: none;"></pre><div class="tt-menu" style="position: absolute; top: 100%; left: 0px; z-index: 100; display: none;"><div class="tt-dataset tt-dataset-instEducativasOtras"></div><div class="tt-dataset tt-dataset-institucionesEducativas"></div></div></span>
</div>
<div class="col-md-6 otroInstitutoWrap">
<label for="otraInstitucion" class="required">Indicá donde estudiaste</label>
<input id="otraInstitucion" name="otraInstitucion" type="text" style="opacity: 1;">
</div>
</div>			        
<div class="col-md-12 pd0 mb15">
<div class="col-md-6">
<label for="rango" class="required">Rango</label>
<div class="select-wrap">
<select id="rango" name="rango" style="opacity: 1;">
<option disabled="" selected="">Seleccionar</option>
<option id="10">desde 1 hasta 10</option>
<option id="11">desde 1 hasta 100</option>
<option id="12">desde A hasta F</option>
<option id="13">desde 1 hasta 7</option>
<option id="14">desde 1 hasta 5</option>
<option id="15">desde 1 hasta 20</option>
</select>
<div class="select-arrow"></div>
</div><!-- select-wrap -->
</div>
<div class="col-md-6">
<label for="promedio" class="required">Promedio</label>
<input id="promedio" name="promedio" type="text" style="opacity: 1;">
</div>
</div>
<a href="" class="btn btn-secondary" id="registrarEducacion">GUARDAR ESTUDIO</a>
<a href="" class="btn btn-empty cancelarForm">CANCELAR</a>
</form>
<div class="clearfix"></div>
</div><!-- /cv-form -->
</div> <!-- /modal-body -->
<!-- <div class="modal-footer">
<button type="button" class="btn btn-default" data-dismiss="modal">Close</button>
<button type="button" class="btn btn-primary">Save changes</button>
</div> -->
</div><!-- /.modal-content -->
</div>
</body>

# ALTAIR Juego api

In [0]:
!pip install altair==2.0.0rc2
import altair as alt
# for colab only run this command once per session
alt.renderers.enable('colab')

In [0]:
import altair as alt
alt.renderers.enable('colab')



In [0]:
1 Horizozntal barplot 
- por nivel educativo vs cantidad . cada color es un estado (abandono, graduado, etc)
. https://altair-viz.github.io/gallery/horizontal_stacked_bar_chart.html
  
2

In [0]:
postulaciones.head(2)

In [0]:
#Agrego columna con DIA de postulacion
postulaciones['day_of_week'] = postulaciones['timestamp_postulacion'].dt.weekday_name
postulaciones.head(2)

In [0]:
df1 = postula

In [0]:
df = postulaciones[['timestamp_postulacion']].dropna()
df['cantidad'] = 1
df

In [0]:
df['fecha_de_postulacion'] = df['timestamp_postulacion'].dt.date
df['horario_de_postulacion'] = df['timestamp_postulacion'].dt.hour
df['dia_de_semana'] = df['timestamp_postulacion'].dt.weekday_name
df

In [0]:
#Agrupo 
df_gb_1 = df[['dia_de_semana', 'horario_de_postulacion', 'cantidad']].groupby(by =['dia_de_semana', 'horario_de_postulacion'], as_index=False).count()

df_gb_1['dia_de_semana'] = (df_gb_1['dia_de_semana'].astype('category').cat.reorder_categories(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],ordered=True))

In [0]:
# Quiero por dia y hora ver la cantidad de postulacion
# https://altair-viz.github.io/gallery/table_bubble_plot_github.html
alt.Chart(df_gb_1).mark_rect().encode(
    alt.Y('dia_de_semana:O'),
    alt.X('horario_de_postulacion:O'),
    size='cantidad:Q', 
    color='cantidad:Q'
    
   
).properties(
    title="Cantidad Postulaciones por dia y hora,"
)

**CONCLUSION** : Vemos que entre las 6 am y 12 am hay mayor cantidad de postulaciones para todos los dias. Los dias Lunes , martes y miercoles se extiende en menor medidad hasta las 21 hs. 

Los fin de semana, las postulaciones decaen, lo cual es logico.

**OBSERVACION** No logramos ordenar por dia cronologicamente. La libreria lo ordena por orden alfabetico :( 

In [0]:
visitas.head(2)

In [0]:
df_2 = visitas[['timestamp_visita']]
df_2['timestamp_visita'] = df_2['timestamp_visita'].dt.floor('h')
df_2['cantidad'] = 1

df_2 = df_2.groupby(by='timestamp_visita',as_index=False).count()
df_2['hora_visita'] = df_2['timestamp_visita'].dt.hour
df_2

In [0]:
line = alt.Chart().mark_line().encode(
    alt.X('timestamp_visita:T'),
    y='cantidad:Q',
    opacity=alt.value(0.3)
    
     )

points = alt.Chart().mark_point().encode(
    alt.X('timestamp_visita:T'),
    y='cantidad:Q',
         )

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['timestamp_visita'], empty='none')


# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart().mark_point().encode(
    x='timestamp_visita:Q',
    opacity=alt.value(0),
).properties(
    selection=nearest
)

# Draw points on the line, and highlight based on selection
# points = line.mark_point().encode(
#     opacity=alt.condition(nearest, alt.value(1), alt.value(0))
# )

#Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'hora_visita:O', alt.value(' '))
)


# Put the five layers into a chart and bind the data
alt.layer(line, selectors, text, points,
          data=df_2, width=600, height=300)

**CONCLUSION** Notamos que hay hay picos entre las 12 y 15 horas. Esto nos da a entender que la gente aprovecha en este tiempo, para hacer nuevas visitas

**PREGUNTAS QUE NOS HACEMOS** 
0. Que edad tienen los visitantes? Para saber si estan estudiando o tienen un trabajo. 
1. Estaran disconformes con sus actuales trabajos?


In [0]:
postulantes.head()

In [0]:
postulantes_genero_edad